# IBM - Applied data science capstone project
This notebook only serves the purpouse to document and pass the capstone project for the data science professional certificate by IBM on coursera. 

Check out the link for more information: [Applied Data Science Capstone](https://www.coursera.org/learn/applied-data-science-capstone)

<br>

## Table of content
1. **[Peer graded assignment 1: Capstone Project Notebook](#PA_1)** <br><br>
2. **[Peer graded Assignment 2: Segmenting and Clustering Neighborhoods in Toronto](#PA_2)** <br>
    2.1 [Task 1: Create a dataframe about Toronto's neighborhoods](#PA_2_1) <br>
    2.2 [Task 2: Add geospatial data to your dataframe](#PA_2_2) <br>
    2.3 [Task 3: Explore and cluster the neighborhoods in Toronto](#PA_2_3)

<br>

## Peer graded assignment 1: Capstone Project Notebook <a name="PA_1" ></a>
The following lines of code are only supposed to satisfy the peer graded assignment number 1 and do not serve any other purpose at this point.

In [100]:
# Import working libraries
import pandas as pd
import numpy as np

In [101]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


<br>
<br>
<br>
<br>

## Peer graded Assignment 2: Segmenting and Clustering Neighborhoods in Toronto <a name="PA_2" ></a>
The following code sections only serve the purpose of solving the given tasks.

<br>

### Task 1: Create a dataframe about Toronto's neighborhoods <a name="PA_2_1" ></a>
To solve this task, we have to scrape the neighborhoods table from Toronto's [Wikipedia website](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M). <br><br>

To do so, we will use the Pandas "read_html" function in order to scrape the Wikipedia page.

In [102]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" # Define url
dfs = pd.read_html(url) # Download tables on website via pandas
print("The downloaded website contains {} tables.".format(len(dfs))) # Display how many tables have been downloaded

The downloaded website contains 3 tables.


As we can see, we have downloaded multiple tables. A quick visual check on the website itself shows us that the information we are interested in lies within table 1. We thus need to extract table 1 from the downloaded website.

In [103]:
df = dfs[0] # Get first table
df.head()

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


We can see that we downloaded the correct data. The data, however, is messy and needs to be cleaned before we can use it for something else:

#### Stacking the columns horizontally
To be able to iterate through the columns more easily, we need to stack them. We can do so by appending each column to a new variable. In result, we will have all converted our initial df with 20 rows and 9 columns to a new df with 180 rows and 1 column.

In [104]:
df_stacked = df[0] # Grab first column in new variable

for column in df.columns[1:]: # Iterate through columns, starting with the second column
    df_stacked = df_stacked.append(df[column]) # Append each column to new variable

df_stacked = pd.DataFrame(df_stacked) # Convert back to df
print("The stacked df has {} rows and {} columns:".format(df_stacked.shape[0], df_stacked.shape[1]))
df_stacked.head()

The stacked df has 180 rows and 1 columns:


,0
0,M1ANot assigned
1,M1BScarborough(Malvern / Rouge)
2,M1CScarborough(Rouge Hill / Port Union / Highl...
3,M1EScarborough(Guildwood / Morningside / West ...
4,M1GScarborough(Woburn)


#### Getting the post codes
When we observe the data more closely, we can notice that the postal code is contained in the first three letters of each string. We can store these in a new column by accessing the first three positions of the string:

In [105]:
df_stacked['POSTAL_CODE'] = df_stacked[0].str[:3] # Receive first three characters as string
df_stacked.head()

,0,POSTAL_CODE
0,M1ANot assigned,M1A
1,M1BScarborough(Malvern / Rouge),M1B
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M1C
3,M1EScarborough(Guildwood / Morningside / West ...,M1E
4,M1GScarborough(Woburn),M1G


#### Getting the boroughs
When we have a second look at the downloaded information, we can see that the boroughs always start after the postal code (first three letters) and end before a "(" character. We can use the split function to split our information and then delete the rest of the string to isolate the boroughs:

In [106]:
split_char = "(" # Define ( as the character to split the string
df_stacked['BOROUGH'] = df_stacked[0].str.split(pat=split_char) # Splits the string after the ( character

df_stacked['BOROUGH'] = df_stacked['BOROUGH'].str[0] # Select first split
df_stacked['BOROUGH'] = df_stacked['BOROUGH'].str[3:] # Kick out the first three letters (postal code)

df_stacked.head()

,0,POSTAL_CODE,BOROUGH
0,M1ANot assigned,M1A,Not assigned
1,M1BScarborough(Malvern / Rouge),M1B,Scarborough
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M1C,Scarborough
3,M1EScarborough(Guildwood / Morningside / West ...,M1E,Scarborough
4,M1GScarborough(Woburn),M1G,Scarborough


#### Getting the neighborhoods
In this step, we need to extract the neighborhoods. We can do that with the same approach as above. Afterwards, we will seperate the neighborhoods with a "," instead of a "/" as the course material tells us to.

In [107]:
split_char = "(" # Define ( as the character to split the string
df_stacked['NEIGHBORHOOD'] = df_stacked[0].str.split(pat=split_char) # Splits the string after the "(" character

df_stacked['NEIGHBORHOOD'] = df_stacked['NEIGHBORHOOD'].str[1] # Select second split
df_stacked['NEIGHBORHOOD'] = df_stacked['NEIGHBORHOOD'].str[:-1] # Kick out the last character "("

df_stacked['NEIGHBORHOOD'] = df_stacked['NEIGHBORHOOD'].str.replace(" / ", ", ")

df_stacked.head()

,0,POSTAL_CODE,BOROUGH,NEIGHBORHOOD
0,M1ANot assigned,M1A,Not assigned,NaN
1,M1BScarborough(Malvern / Rouge),M1B,Scarborough,"Malvern, Rouge"
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1EScarborough(Guildwood / Morningside / West ...,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1GScarborough(Woburn),M1G,Scarborough,Woburn


#### Cleaning up the dataframe
Finally, we can clean up the data to fullfill the courses' requirements for task 1. First, we will drop the column with the messy data:

In [108]:
df_stacked.drop(columns=[0], inplace=True)
df_stacked.head()

,POSTAL_CODE,BOROUGH,NEIGHBORHOOD
0,M1A,Not assigned,NaN
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


Next, we will get rid of all rows where the borough is not assigned:

In [109]:
df_stacked = df_stacked.reset_index(drop=True) # Reset index
index_names = df_stacked[df_stacked['BOROUGH'] == 'Not assigned'].index # Define condition
df_stacked.drop(index_names, axis=0, inplace=True) # Drop rows that meet condition
df_stacked.head()

,POSTAL_CODE,BOROUGH,NEIGHBORHOOD
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


The course says that the data set includes duplicates in the postal codes column. We therefore will check our dataframe for duplicates: 

In [110]:
duplicates = df_stacked['POSTAL_CODE'].duplicated() # Check for duplicates
print("We have {} duplicates in our postal codes.".format(duplicates.sum())) # Count duplicates = True

We have 0 duplicates in our postal codes.


As we have no duplicates in our postal code column, we can move on and check whether we have a the value "Not assigned" in our neighborhoods column:

In [111]:
if 'Not assigned' in set(df_stacked['NEIGHBORHOOD']) == True:
    print("There are missing neighborhoods, please check.")

else:
    print("No missing neighborhoods, proceed.")

No missing neighborhoods, proceed.


Since we do not have any unassigned neighborhoods, we can proceed to the final step - assigning our cleaned dataframe to a new variable for further processing:

In [112]:
toronto = df_stacked
print("The Torono neighborhood df has {} rows and {} columns:".format(toronto.shape[0], toronto.shape[1]))
print()
toronto.head()

The Torono neighborhood df has 103 rows and 3 columns:



,POSTAL_CODE,BOROUGH,NEIGHBORHOOD
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


<br>

### Task 2: Add geospatial data to your dataframe <a name="PA_2_2" ></a>
To solve this task we are asked to add geographical data to our toronto dataframe. We are asked to do so using the geocoder library. As the code provided by the course includes an infinite while loop that does not give us a guarantee for success, I have decided to intially run it only for 100 times. I will then decide how to proceed based on the results: 

In [113]:
import geocoder

for i in range(100):
   g = geocoder.google('{}, Toronto, Ontario'.format('M1B'))
   lat_lng_coords = g.latlng
   print(lat_lng_coords)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


We can see that the code returns 100 times "None". We can check the variable "g" for more information: 

In [114]:
print(g)

<[REQUEST_DENIED] Google - Geocode [empty]>


As we can see, the request was denied by Google. A quick research regarding the API shows that the geocoder library nolonger seems to work as Google has provided its own [Google Maps library](https://github.com/googlemaps/google-maps-services-python). The google maps library requires an API key for geocoding which can be purchased [here](https://console.cloud.google.com/apis/library/geocoding-backend.googleapis.com?project=ibm-capstone-projekt). We will refrain from doing so as the information has been provided by the course for free. After downloading, we can import the csv file to a df via Pandas:

In [115]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
print("The dataset has {} rows and {} columns".format(coordinates.shape[0], coordinates.shape[1]))
coordinates.head()

The dataset has 103 rows and 3 columns


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We can then merge the imported dataframe with our Toronto data set:

In [116]:
# Merge dfs
toronto = toronto.merge(
    coordinates, # New information
    how='left', # Make sure to keep everything from original df
    left_on='POSTAL_CODE', # Left key
    right_on='Postal Code') # Right key

# Drop duplicated column
toronto.drop(columns="Postal Code", inplace=True)

# Display df
print("The dataset has {} rows and {} columns".format(toronto.shape[0], toronto.shape[1]))
toronto.head()

The dataset has 103 rows and 5 columns


,POSTAL_CODE,BOROUGH,NEIGHBORHOOD,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<br>

### Task 3: Explore and cluster the neighborhoods in Toronto <a name="PA_2_3" ></a>
For this task, we are pretty much free to do whatever we want. The only three requirements are: <br>
<br>
1. The analysis should have the same analysis depth as the New York excercise.<br>
2. The notebook have sufficient markdown to explain the conducted analysis<br>
3. The analysis needs to include sufficient maps to visualize the clusters. 